In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [15]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = 0.2*init.kaiming_uniform_(torch.empty(d1, d0, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.kaiming_uniform_(torch.empty(d2, d1, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.kaiming_uniform_(torch.empty(d3, d2, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.kaiming_uniform_(torch.empty(d4, d3, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = 0.2*init.kaiming_uniform_(torch.empty(d5, d4, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "KaimingUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 2.6187729835510254 - sq_loss: 661.7448120117188 - tot_loss: 1252.5230238315999 - acc: 0.19423286180631122 - val_acc: 0.19511367492365117
Repeatition 1 Epoch 2 / 500 
 - time: 2.6186187267303467 - sq_loss: 294.10882568359375 - tot_loss: 515.7608848810196 - acc: 0.25571273122959737 - val_acc: 0.23922633186291142
Repeatition 1 Epoch 3 / 500 
 - time: 2.5946764945983887 - sq_loss: 157.2239227294922 - tot_loss: 292.48887702450156 - acc: 0.47429270946681173 - val_acc: 0.45877163216830674
Repeatition 1 Epoch 4 / 500 
 - time: 2.5693840980529785 - sq_loss: 84.50540924072266 - tot_loss: 173.97780356928706 - acc: 0.5410772578890098 - val_acc: 0.5300305395317272
Repeatition 1 Epoch 5 / 500 
 - time: 2.616443395614624 - sq_loss: 45.35883331298828 - tot_loss: 108.56599060632288 - acc: 0.5877312295973884 - val_acc: 0.5744825246012895
Repeatition 1 Epoch 6 / 500 
 - time: 

Repeatition 1 Epoch 48 / 500 
 - time: 2.5994873046875 - sq_loss: 0.000723005214240402 - tot_loss: 2.079909611140465 - acc: 0.8903699673558215 - val_acc: 0.8306752629793009
Repeatition 1 Epoch 49 / 500 
 - time: 2.544754981994629 - sq_loss: 0.0006841194117441773 - tot_loss: 1.9824618332895625 - acc: 0.8928182807399347 - val_acc: 0.832371903630811
Repeatition 1 Epoch 50 / 500 
 - time: 2.5071349143981934 - sq_loss: 0.0006476641283370554 - tot_loss: 1.9072562521905638 - acc: 0.8936343852013058 - val_acc: 0.832371903630811
Repeatition 1 Epoch 51 / 500 
 - time: 2.5887444019317627 - sq_loss: 0.0006137029267847538 - tot_loss: 1.8195157664267754 - acc: 0.8948585418933623 - val_acc: 0.832371903630811
Repeatition 1 Epoch 52 / 500 
 - time: 2.58123779296875 - sq_loss: 0.0005819800426252186 - tot_loss: 1.7568441487528617 - acc: 0.8956746463547334 - val_acc: 0.833050559891415
Repeatition 1 Epoch 53 / 500 
 - time: 2.50972318649292 - sq_loss: 0.0005522909923456609 - tot_loss: 1.7074046515444934 - 

Repeatition 1 Epoch 95 / 500 
 - time: 2.551281452178955 - sq_loss: 9.348502499051392e-05 - tot_loss: 0.4982248112855814 - acc: 0.9362078346028292 - val_acc: 0.8849677638276213
Repeatition 1 Epoch 96 / 500 
 - time: 2.556293249130249 - sq_loss: 9.043748286785558e-05 - tot_loss: 0.4869306413738741 - acc: 0.9368879216539717 - val_acc: 0.8859857482185273
Repeatition 1 Epoch 97 / 500 
 - time: 2.576005458831787 - sq_loss: 8.749637345317751e-05 - tot_loss: 0.48866199233589214 - acc: 0.9379760609357998 - val_acc: 0.8859857482185273
Repeatition 1 Epoch 98 / 500 
 - time: 2.5220754146575928 - sq_loss: 8.469400927424431e-05 - tot_loss: 0.4775728924651048 - acc: 0.9389281828073993 - val_acc: 0.8876823888700374
Repeatition 1 Epoch 99 / 500 
 - time: 2.5323872566223145 - sq_loss: 8.199943113140762e-05 - tot_loss: 0.4712616969713963 - acc: 0.9392002176278563 - val_acc: 0.8890397013912453
Repeatition 1 Epoch 100 / 500 
 - time: 2.5023891925811768 - sq_loss: 7.946552796056494e-05 - tot_loss: 0.475301

Repeatition 1 Epoch 141 / 500 
 - time: 2.5142879486083984 - sq_loss: 3.089355959673412e-05 - tot_loss: 0.25617382788311716 - acc: 0.9644994559303591 - val_acc: 0.9158466236851035
Repeatition 1 Epoch 142 / 500 
 - time: 2.477722644805908 - sq_loss: 3.0444347430602647e-05 - tot_loss: 0.2660640091816049 - acc: 0.9650435255712732 - val_acc: 0.9158466236851035
Repeatition 1 Epoch 143 / 500 
 - time: 2.501218318939209 - sq_loss: 2.9991622795932926e-05 - tot_loss: 0.2472197022783007 - acc: 0.9657236126224157 - val_acc: 0.9168646080760094
Repeatition 1 Epoch 144 / 500 
 - time: 2.482121229171753 - sq_loss: 2.9554601496784016e-05 - tot_loss: 0.2620868950334625 - acc: 0.9658596300326442 - val_acc: 0.9175432643366135
Repeatition 1 Epoch 145 / 500 
 - time: 2.5501441955566406 - sq_loss: 2.913915886892937e-05 - tot_loss: 0.25384419879105735 - acc: 0.9658596300326442 - val_acc: 0.9178825924669155
Repeatition 1 Epoch 146 / 500 
 - time: 2.488529920578003 - sq_loss: 2.87296024907846e-05 - tot_loss: 0

Repeatition 1 Epoch 187 / 500 
 - time: 2.595858097076416 - sq_loss: 1.9247278032707982e-05 - tot_loss: 0.2131174661114983 - acc: 0.9752448313384113 - val_acc: 0.9290804207668816
Repeatition 1 Epoch 188 / 500 
 - time: 2.545872688293457 - sq_loss: 1.9124863683828153e-05 - tot_loss: 0.19724904207009786 - acc: 0.9753808487486398 - val_acc: 0.9297590770274856
Repeatition 1 Epoch 189 / 500 
 - time: 2.5785019397735596 - sq_loss: 1.9007826267625205e-05 - tot_loss: 0.19539326023675585 - acc: 0.9760609357997824 - val_acc: 0.9300984051577876
Repeatition 1 Epoch 190 / 500 
 - time: 2.6041977405548096 - sq_loss: 1.8879856725106947e-05 - tot_loss: 0.18838444090340545 - acc: 0.9761969532100109 - val_acc: 0.9304377332880895
Repeatition 1 Epoch 191 / 500 
 - time: 2.6249401569366455 - sq_loss: 1.8755548808258027e-05 - tot_loss: 0.1894080545804968 - acc: 0.9764689880304679 - val_acc: 0.9304377332880895
Repeatition 1 Epoch 192 / 500 
 - time: 2.556164264678955 - sq_loss: 1.8635717424331233e-05 - tot_l

...

Repeatition 1 Epoch 279 / 500 
 - time: 1.2816615104675293 - sq_loss: 1.3102246157359332e-05 - tot_loss: 0.17374557844661354 - acc: 0.985038084874864 - val_acc: 0.9460468272819816
Repeatition 1 Epoch 280 / 500 
 - time: 1.273874282836914 - sq_loss: 1.306357626162935e-05 - tot_loss: 0.18366220123303378 - acc: 0.9851741022850925 - val_acc: 0.9460468272819816
Repeatition 1 Epoch 281 / 500 
 - time: 1.2999005317687988 - sq_loss: 1.3021602171647828e-05 - tot_loss: 0.17374881558829713 - acc: 0.9854461371055495 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 282 / 500 
 - time: 1.297987699508667 - sq_loss: 1.2982389307580888e-05 - tot_loss: 0.18220971712911194 - acc: 0.9854461371055495 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 283 / 500 
 - time: 1.2826886177062988 - sq_loss: 1.2951184544363059e-05 - tot_loss: 0.1657788876509585 - acc: 0.985582154515778 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 284 / 500 
 - time: 1.3011033535003662 - sq_loss: 1.2919590517412871e-05 - tot_lo

Repeatition 1 Epoch 325 / 500 
 - time: 1.2930433750152588 - sq_loss: 1.1643323887255974e-05 - tot_loss: 0.16286070109336492 - acc: 0.9863982589771491 - val_acc: 0.9504580929759077
Repeatition 1 Epoch 326 / 500 
 - time: 1.300661563873291 - sq_loss: 1.1612994967435952e-05 - tot_loss: 0.16434597051565447 - acc: 0.9863982589771491 - val_acc: 0.9507974211062097
Repeatition 1 Epoch 327 / 500 
 - time: 1.2887482643127441 - sq_loss: 1.1582174920476973e-05 - tot_loss: 0.1647199819516203 - acc: 0.9865342763873776 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 328 / 500 
 - time: 1.2687122821807861 - sq_loss: 1.1551743227755651e-05 - tot_loss: 0.1717875868669978 - acc: 0.9868063112078346 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 329 / 500 
 - time: 1.2995283603668213 - sq_loss: 1.1524231922521722e-05 - tot_loss: 0.17611622674931482 - acc: 0.9866702937976061 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 330 / 500 
 - time: 1.283977746963501 - sq_loss: 1.1495561921037734e-05 - tot_

Repeatition 1 Epoch 371 / 500 
 - time: 1.304398775100708 - sq_loss: 1.0516596375964582e-05 - tot_loss: 0.15099881582037966 - acc: 0.9877584330794341 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 372 / 500 
 - time: 1.288930892944336 - sq_loss: 1.049567072186619e-05 - tot_loss: 0.14858608250090555 - acc: 0.9876224156692056 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 373 / 500 
 - time: 1.2982566356658936 - sq_loss: 1.0472665962879546e-05 - tot_loss: 0.1619532894226836 - acc: 0.9877584330794341 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 374 / 500 
 - time: 1.2764670848846436 - sq_loss: 1.045116368914023e-05 - tot_loss: 0.14066788620206694 - acc: 0.9878944504896626 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 375 / 500 
 - time: 1.2843780517578125 - sq_loss: 1.0431339433125686e-05 - tot_loss: 0.16599018629719353 - acc: 0.9876224156692056 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 376 / 500 
 - time: 1.2639386653900146 - sq_loss: 1.0412255505798385e-05 - tot_l

Repeatition 1 Epoch 417 / 500 
 - time: 1.2837440967559814 - sq_loss: 9.63877209869679e-06 - tot_loss: 0.16816268955456337 - acc: 0.9889825897714908 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 418 / 500 
 - time: 1.3040742874145508 - sq_loss: 9.622707693779375e-06 - tot_loss: 0.16979439677967179 - acc: 0.9889825897714908 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 419 / 500 
 - time: 1.2852671146392822 - sq_loss: 9.608618711354211e-06 - tot_loss: 0.1586435266544015 - acc: 0.9892546245919478 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 420 / 500 
 - time: 1.3003015518188477 - sq_loss: 9.593235517968424e-06 - tot_loss: 0.1450319126630646 - acc: 0.9891186071817193 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 421 / 500 
 - time: 1.3070805072784424 - sq_loss: 9.573559509590268e-06 - tot_loss: 0.15648986651807206 - acc: 0.9891186071817193 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 422 / 500 
 - time: 1.3059697151184082 - sq_loss: 9.553603376843967e-06 - tot_loss:

Repeatition 1 Epoch 463 / 500 
 - time: 1.29811429977417 - sq_loss: 8.903413799998816e-06 - tot_loss: 0.1533646807023672 - acc: 0.9897986942328618 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 464 / 500 
 - time: 1.2982990741729736 - sq_loss: 8.891728612070438e-06 - tot_loss: 0.17389367136237865 - acc: 0.9897986942328618 - val_acc: 0.9596199524940617
Repeatition 1 Epoch 465 / 500 
 - time: 1.3047261238098145 - sq_loss: 8.877788786776364e-06 - tot_loss: 0.17878630926004746 - acc: 0.9896626768226333 - val_acc: 0.9599592806243638
Repeatition 1 Epoch 466 / 500 
 - time: 1.2989721298217773 - sq_loss: 8.865770723787136e-06 - tot_loss: 0.16221111285408085 - acc: 0.9896626768226333 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 467 / 500 
 - time: 1.298570156097412 - sq_loss: 8.849533514876384e-06 - tot_loss: 0.167150726659834 - acc: 0.9896626768226333 - val_acc: 0.9602986087546658
Repeatition 1 Epoch 468 / 500 
 - time: 1.280057430267334 - sq_loss: 8.833619176584762e-06 - tot_loss: 0.1